In [2]:
# 图的存储：临接表、临接矩阵
# 图的遍历：所有可能路径，DAG
# dfs遍历，关注的是节点
graph = [[4, 3, 1], [3, 2, 4],[3], [4], []]

def allPathsSourceTarget(graph):
    def traverse(graph, s, path):
        # 添加节点s到路径
        path.append(s)
        
        # 到达终点
        if s == len(graph) - 1:
            res.append(path[:])

        # 递归遍历每个相邻节点
        for v in graph[s]:
            traverse(graph, v, path)

        # 从路径中移除节点s，s此时在path的末尾
        path.pop()
        
    res = []
    traverse(graph, 0, [])
    return res
allPathsSourceTarget(graph)

[[0, 4], [0, 3, 4], [0, 1, 3, 4], [0, 1, 2, 3, 4], [0, 1, 4]]

In [3]:
# 回溯法遍历，关注的是树枝
graph = [[4, 3, 1], [3, 2, 4],[3], [4], []]

def allPathsSourceTarget(graph):
    def traceback(graph, node, path, target):
        # 结束条件，到达目标节点
        if node == target:
            res.append(path[:])
            return
        
        # 选择列表是当前node的所有邻居节点
        for neighbor in graph[node]:
            # 做选择
            path.append(neighbor)
            # 递归
            traceback(graph, neighbor, path, target)
            # 撤销选择
            path.pop()
            
    res = []
    traceback(graph, 0, [0], len(graph) - 1)
    return res

allPathsSourceTarget(graph)

[[0, 4], [0, 3, 4], [0, 1, 3, 4], [0, 1, 2, 3, 4], [0, 1, 4]]

In [4]:
# BFS遍历
graph = [[4, 3, 1], [3, 2, 4],[3], [4], []]

import collections

def allPathsSourceTarget(graph):
    res = []
    # 使用队列完成BFS，将路径入队，初始路径是[0]
    q = collections.deque([[0]])
    while q:
        path = q.popleft()
        # 取path的最后一个元素，看是不是目标len(graph) - 1
        if path[-1] == len(graph) - 1:
            # 此路径已经完成，加入到结果
            res.append(path[:])
        else:
            # 未完成，遍历path最后一个元素的邻居节点
            for neighbor in graph[path[-1]]:
                # 将原path拼接到当前邻居得到新路径入队
                q.append(path + [neighbor])
    return res

allPathsSourceTarget(graph)

[[0, 4], [0, 3, 4], [0, 1, 4], [0, 1, 3, 4], [0, 1, 2, 3, 4]]

In [5]:
# 环检测算法，DFS、BFS
# leetcode 207 课程表
from collections import deque

class Solution:
    # DFS
    def canFinish(self, numCourses: int, prerequisites) -> bool:
        # 先根据依赖关系建图
        graph = [[] for _ in range(numCourses)]
        for edge in prerequisites:
            pre = edge[1]
            cur = edge[0]
            # 建立课程从pre到to的依赖
            graph[pre].append(cur)
        
        # 记录当前 traverse 经过的路径
        onpath = [False for _ in range(numCourses)]
        # 标记节点被遍历的情况
        visited = [False for _ in range(numCourses)]
        self.hascycle = False

        # DFS遍历函数
        def traverse(graph, v):
            # 如果当前traverse经过的路径已经遍历过了 说明有环
            if onpath[v]:
                self.hascycle = True
            if visited[v] or self.hascycle:
                # 如果节点已经被遍历或找到了环 就不继续遍历了
                return
            # 标记节点v为已访问
            visited[v] = True
            # 前序位置 标记进入节点v的遍历
            onpath[v] = True
            for neighbor in graph[v]:
                traverse(graph, neighbor)
            # 后序位置 离开节点v的遍历
            onpath[v] = False
        
        # 由于图可能不是全联通的 所以需要对每个节点都调用一次遍历函数
        for i in range(numCourses):
            # 遍历每个节点
            traverse(graph, i)
        
        # 只要没有循环依赖（环）就说明可以完成所有课程，否则不能
        return not self.hascycle
    
    # BFS
    def canFinish(self, numCourses: int, prerequisites) -> bool:
        indegrees = [0 for _ in range(numCourses)]
        adjacency = [[] for _ in range(numCourses)]
        
        queue = deque()
        for cur, pre in prerequisites:
            # 统计每个节点的入度
            indegrees[cur] += 1
            # 得到图的邻接表
            adjacency[pre].append(cur)
        
        for i in range(len(indegrees)):
            # 将所有入度为 0 的节点入队
            if not indegrees[i]:
                queue.append(i)
        
        # 当 queue 非空时，依次将队首节点出队，在课程安排图中删除此节点 pre
        while queue:
            pre = queue.popleft()
            numCourses -= 1
            for cur in adjacency[pre]:
                indegrees[cur] -= 1
                # cur 所有的前驱节点已经被删除
                if not indegrees[cur]:
                    queue.append(cur)
        # 若课程安排图中存在环，一定有节点的入度始终不为 0
        return not numCourses

In [6]:
# 拓扑排序算法，DFS、BFS
# 拓扑排序原理： 对 DAG 的顶点进行排序，使得对每一条有向边 (u,v)，均有 u（在排序记录中）比 v 先出现。亦可理解为对某点 v 而言，只有当 v 的所有源点均出现了，v 才能出现。
# 如果一幅图是「有向无环图」，那么一定可以进行拓扑排序
# leetcode 210 课程表 II
from typing import List

class Solution:
    def findOrder(self, numCourses: int, prerequisites: List[List[int]]) -> List[int]:
        # 存储有向图
        edges = collections.defaultdict(list)
        # 标记每个节点的状态：0=未搜索，1=搜索中，2=已完成
        visited = [0] * numCourses
        # 用数组来模拟栈，下标 0 为栈底，n-1 为栈顶
        result = list()
        # 判断有向图中是否有环
        valid = True

        for info in prerequisites:
            edges[info[1]].append(info[0])
        
        def dfs(u: int):
            nonlocal valid
            # 将节点标记为「搜索中」
            visited[u] = 1
            # 搜索其相邻节点
            # 只要发现有环，立刻停止搜索
            for v in edges[u]:
                # 如果「未搜索」那么搜索相邻节点
                if visited[v] == 0:
                    dfs(v)
                    if not valid:
                        return
                # 如果「搜索中」说明找到了环
                elif visited[v] == 1:
                    valid = False
                    return
            # 将节点标记为「已完成」
            visited[u] = 2
            # 将节点入栈
            result.append(u)
        
        # 每次挑选一个「未搜索」的节点，开始进行深度优先搜索
        for i in range(numCourses):
            if valid and not visited[i]:
                dfs(i)
        
        if not valid:
            return list()
        
        # 如果没有环，那么就有拓扑排序
        # 注意下标 0 为栈底，因此需要将数组反序输出
        return result[::-1]

    
    # BFS
    def findOrder(self, numCourses: int, prerequisites: List[List[int]]) -> List[int]:
        indegrees = [0 for _ in range(numCourses)]  # 入读表
        adjacency = [[] for _ in range(numCourses)]  # 邻接矩阵
        res = []
        queue = []
        for cur, pre in prerequisites:
            indegrees[cur] += 1  # 得到入度
            adjacency[pre].append(cur)  # 得到每个课程节点的邻接表
        for i in range(len(indegrees)):
            if not indegrees[i]:
                queue.append(i)  # 把度为0的课程入队
        while queue:
            pre = queue.pop(0)
            res.append(pre)
            numCourses -= 1  # 每次pre出队时，numCourses减1
            for cur in adjacency[pre]:
                indegrees[cur] -= 1  # 将这个节点的入度减1
                if not indegrees[cur]:  # cur的所有的前驱节点已经被删除，cur入队
                    queue.append(cur)
        return res if not numCourses else []  # 拓扑排序出队次数等于课程个数，课程安排图是否是有向无环图(DAG)，课程可以成功安排

In [7]:
# 二分图判定算法，DFS、BFS
# leetcode 785 判断二分图

# dfs
class Solution:
    def isBipartite(self, graph):
        n = len(graph)
        UNCOLORED, RED, GREEN = 0, 1, 2
        color = [UNCOLORED] * n
        valid = True

        def dfs(node, c):
            nonlocal valid
            color[node] = c
            cNei = (GREEN if c == RED else RED)
            for neighbor in graph[node]:
                if color[neighbor] == UNCOLORED:  # 未染色
                    dfs(neighbor, cNei)
                    if not valid:
                        return
                elif color[neighbor] != cNei:  # 已染色，但不对
                    valid = False
                    return

        for i in range(n):
            if color[i] == UNCOLORED:
                dfs(i, RED)
                if not valid:
                    break

        return valid

# bfs
class Solution:
    def isBipartite(self, graph):
        n = len(graph)
        UNCOLORED, RED, GREEN = 0, 1, 2
        color = [UNCOLORED] * n

        for i in range(n):
            if color[i] == UNCOLORED:
                q = collections.deque([i])
                color[i] = RED
                while q:
                    node = q.popleft()
                    cNei = (GREEN if color[node] == RED else RED)
                    for neighbor in graph[node]:
                        if color[neighbor] == UNCOLORED: # 未染色
                            q.append(neighbor)
                            color[neighbor] = cNei
                        elif color[neighbor] != cNei: # 已染色，但不对
                            return False
        return True

In [8]:
# 并查集算法
# leetcode 130 被围绕的区域
from typing import List


class Solution(object):
    def solve_DFS(self, board):
        """
        :type board: List[List[str]]
        :rtype: None Do not return anything, modify board in-place instead.
        """
        if not board or not board[0]:
            return
        row = len(board)
        col = len(board[0])

        def dfs(i, j):
            board[i][j] = "B"
            for x, y in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                tmp_i = i + x
                tmp_j = j + y
                if 1 <= tmp_i < row and 1 <= tmp_j < col and board[tmp_i][tmp_j] == "O":
                    dfs(tmp_i, tmp_j)

        # 从边界出发，把边界上和'O'连通点变为'B'
        for j in range(col):
            # 第一行
            if board[0][j] == "O":
                dfs(0, j)
            # 最后一行
            if board[row - 1][j] == "O":
                dfs(row - 1, j)

        for i in range(row):
            # 第一列
            if board[i][0] == "O":
                dfs(i, 0)
            # 最后一列
            if board[i][col - 1] == "O":
                dfs(i, col - 1)

        # 再遍历整个board，将'O'变为'X'，将'B'变为'O'
        for i in range(row):
            for j in range(col):
                # O 变成 X
                if board[i][j] == "O":
                    board[i][j] = "X"
                # B 变成 O
                if board[i][j] == "B":
                    board[i][j] = "O"

    def solve_BFS(self, board):
        if not board or not board[0]:
            return
        row = len(board)
        col = len(board[0])

        def bfs(i, j):
            from collections import deque
            queue = deque()
            queue.appendleft((i, j))
            while queue:
                i, j = queue.pop()
                if 0 <= i < row and 0 <= j < col and board[i][j] == "O":
                    board[i][j] = "B"
                    for x, y in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                        queue.appendleft((i + x, j + y))

        # 从边界出发，把边界上和'O'连通点变为'B'
        for j in range(col):
            # 第一行
            if board[0][j] == "O":
                bfs(0, j)
            # 最后一行
            if board[row - 1][j] == "O":
                bfs(row - 1, j)

        for i in range(row):

            if board[i][0] == "O":
                bfs(i, 0)
            if board[i][col - 1] == "O":
                bfs(i, col - 1)

        # 再遍历整个board，将'O'变为'X'，将'B'变为'O'
        for i in range(row):
            for j in range(col):
                if board[i][j] == "O":
                    board[i][j] = "X"
                if board[i][j] == "B":
                    board[i][j] = "O"

    def solve_UnionFindSet(self, board):
        f = {}

        # 返回某个节点的根节点
        def find(x):
            f.setdefault(x, x)
            if f[x] != x:
                f[x] = find(f[x])
            return f[x]

        # 连接两个节点
        def union(x, y):
            f[find(y)] = find(x)

        if not board or not board[0]:
            return
        row = len(board)
        col = len(board[0])
        dummy = row * col


        for i in range(row):
            for j in range(col):
                if board[i][j] == "O":
                    if i == 0 or i == row - 1 or j == 0 or j == col - 1:    # 将四周的'O'和dummy节点相连
                        union(i * col + j, dummy)
                    else:
                        for x, y in [(-1, 0), (1, 0), (0, -1), (0, 1)]:  # 区域内的'O'和相邻的'O'节点相连
                            if board[i + x][j + y] == "O":
                                union(i * col + j, (i + x) * col + (j + y))
        for i in range(row):
            for j in range(col):
                if find(dummy) == find(i * col + j):
                    board[i][j] = "O"
                else:
                    board[i][j] = "X"